In [144]:
import pandas as pd
import numpy as np
import datetime
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split

In [145]:
data = pd.read_csv('../Documents/HighRadius/1806077.csv', nrows=50000)
data.shape

(50000, 19)

In [146]:
data['clear_date']=pd.to_datetime(data.clear_date)
data['due_in_date']=pd.to_datetime(data.due_in_date,format="%Y%m%d")

In [147]:
data['delay']=data['clear_date']-data['due_in_date']

In [148]:
X_test=data[data['clear_date'].isna()]
X_train=data[data['clear_date'].notna()]

In [149]:
target_mapper = X_train.groupby('name_customer')['delay'].agg(lambda x:x.value_counts().index[0]).to_dict()

In [150]:
def addX_train(X_train):
    X_train.drop(['posting_id', 'area_business'],axis=1,inplace=True)
    X_train['invoice_id'] = X_train['invoice_id'].fillna(X_train['doc_id'])
    X_train['posting_date']=pd.to_datetime(X_train.posting_date,format="%Y-%m-%d")
    X_train['baseline_create_date']=pd.to_datetime(X_train.baseline_create_date,format="%Y%m%d")
    X_train['document_create_date.1']=pd.to_datetime(X_train['document_create_date.1'],format="%Y%m%d")
    X_train['delay']=( X_train['delay']/ np.timedelta64 (1, 'D')).astype (float)
    Y_train=X_train['delay']
    X_train.drop('document_create_date',axis=1,inplace=True)
    X_train=X_train.sort_values(by="document_create_date.1")
    X_train['cube_root_open_amount']=np.cbrt(X_train['total_open_amount'])
    mapper = X_train.groupby('name_customer')['total_open_amount'].mean().to_dict() 
    X_train['avg_amt_company'] = X_train['name_customer'].map(mapper)
    target_mapper = X_train.groupby('name_customer')['delay'].agg(lambda x:x.value_counts().index[0]).to_dict()
    X_train['modedelay'] = X_train['name_customer'].map(target_mapper)
    bin_ranges=[-90,0,10,20,30,50,100,150,200]
    bin_names=[-1,0,1,2,3,4,5,6]
    X_train['delay_bins'] = pd.cut( np.array(X_train['modedelay']),bins=bin_ranges, labels=bin_names)
    X_train['diff']=X_train['due_in_date']-X_train['document_create_date.1']
    X_train['diff']=( X_train['diff']/ np.timedelta64 (1, 'D')).astype (float)
    X_train['whenlate']=np.where(X_train['delay_bins'] >=1,X_train['diff'],0) 
    X_train['duemonth']=X_train['due_in_date'].dt.month
    X_train['dueday']=X_train['due_in_date'].dt.day
    X_train['dueyear']=X_train['due_in_date'].dt.year
    X_train['paytime']=X_train['due_in_date']+pd.to_timedelta(X_train['modedelay'])
    X_train['paytime']=X_train['paytime']-X_train['document_create_date.1']
    X_train['paytime']=( X_train['paytime']/ np.timedelta64 (1, 'D')).astype (float)
    X_train['late']=np.where(X_train['delay_bins'] >=1,1,0)
    X_train['ontime']=np.where(X_train['delay_bins'] <= 0,1,0)
    mapper = X_train.groupby('name_customer')['late'].sum().to_dict()
    X_train['totallate'] = X_train['name_customer'].map(mapper)
    X_train['delay_bins'].fillna(0,inplace=True)
    X_train['delay_bins']=X_train.delay_bins.astype('category').cat.codes
    X_train.drop(['ontime', 'dueyear', 'paytime', 'invoice_id'],axis=1,inplace=True)
    obj_columns=list(X_train.columns[X_train.dtypes=='object'])
    X_train.drop(obj_columns,axis=1,inplace=True)
    date_columns=list(X_train.columns[X_train.dtypes=='datetime64[ns]'])
    X_train.drop(date_columns,axis=1,inplace=True)
    X_train.drop('delay',axis=1,inplace=True)
    return X_train,Y_train

In [151]:
def addX_test(X_test,target_mapper):
    X_test['modedelay'] = X_test['name_customer'].map(target_mapper)
    X_test['modedelay']=( X_test['modedelay']/ np.timedelta64 (1, 'D')).astype (float)
    X_test['modedelay'].fillna(0,inplace=True)
    X_test['cube_root_open_amount']=np.cbrt(X_test['total_open_amount'])
    mapper = X_test.groupby('name_customer')['total_open_amount'].mean().to_dict() 
    X_test['avg_amt_company'] = X_test['name_customer'].map(mapper)
    bin_ranges=[-90,0,10,20,30,50,100,150,200]
    bin_names=[-1,0,1,2,3,4,5,6]
    X_test['delay_bins'] = pd.cut( np.array(X_test['modedelay']),bins=bin_ranges, labels=bin_names)
    X_test['posting_date']=pd.to_datetime(X_test.posting_date,format="%Y-%m-%d")
    X_test['baseline_create_date']=pd.to_datetime(X_test.baseline_create_date,format="%Y%m%d")
    X_test['document_create_date.1']=pd.to_datetime(X_test['document_create_date.1'],format="%Y%m%d")
    X_test['diff']=X_test['due_in_date']-X_test['document_create_date.1']
    X_test['diff']=( X_test['diff']/ np.timedelta64 (1, 'D')).astype (float)
    X_test['whenlate']=np.where(X_test['delay_bins'] >=1,X_test['diff'],0) 
    X_test['duemonth']=X_test['due_in_date'].dt.month
    X_test['dueday']=X_test['due_in_date'].dt.day
    X_test['dueyear']=X_test['due_in_date'].dt.year
    X_test['paytime']=X_test['due_in_date']+pd.to_timedelta(X_test['modedelay'],'D')
    X_test['paytime']=X_test['paytime']-X_test['document_create_date.1']
    X_test['paytime']=( X_test['paytime']/ np.timedelta64 (1, 'D')).astype (float)
    X_test['late']=np.where(X_test['delay_bins'] >=1,1,0)
    X_test['ontime']=np.where(X_test['delay_bins'] <= 0,1,0)
    mapper = X_test.groupby('name_customer')['late'].sum().to_dict()
    X_test['totallate'] = X_test['name_customer'].map(mapper)
    obj_columns=list(X_test.columns[X_test.dtypes=='object'])
    date_columns=list(X_test.columns[X_test.dtypes=='datetime64[ns]'])
    X_test.drop(obj_columns,axis=1,inplace=True)
    X_test.drop(date_columns,axis=1,inplace=True)
    X_test.drop(['ontime', 'dueyear', 'paytime', 'invoice_id'],axis=1,inplace=True)
    X_test['delay_bins'].fillna(0,inplace=True)
    X_test['delay_bins']=X_test.delay_bins.astype('category').cat.codes
    X_test.drop(['delay'],axis=1,inplace=True)
    X_test.drop(['document_create_date','posting_id','area_business'],axis=1,inplace=True)
    return X_test;

In [152]:
def fitmodeltest(X_train,Y_train,X_test):
    from sklearn.ensemble import RandomForestRegressor
    clf = RandomForestRegressor()
    clf.fit(X_train, Y_train)

    # Predicting the Test Set Results
    predicted = clf.predict(X_test)
    return predicted

In [153]:
def fitmodel(X_train,X_test):
    X_train,Y_train=addX_train(X_train)
    X_test=addX_test(X_test,target_mapper)
    predicted=fitmodeltest(X_train,Y_train,X_test)
    return predicted,Y_train

In [154]:
predicted,Y_train=fitmodel(X_train,X_test)

In [155]:
def merge():
    Lastset=data[data['clear_date'].isna()]
    Lastset['delay']=np.round(predicted)
    Lastset['delay']=pd.to_timedelta(Lastset['delay'], unit='D', errors='raise')
    Lastset['clear_date']=Lastset['due_in_date']+Lastset['delay']
    data[data['clear_date'].isna()]=Lastset
    data['delay']=( data['delay']/ np.timedelta64 (1, 'D')).astype (float)
    bin_ranges=[-90,0,15,30,60,90,120,150,180]
    bin_names=['<0','0-15','16-30','31-60','60-90','90-120','121-150','>180']
    data['AgingBucket'] = pd.cut( np.array(data['delay']),bins=bin_ranges, labels=bin_names)
    
    return data

In [156]:
data=merge()

In [157]:
data.to_csv('invoice.csv')